In [1]:
import sys
import os
import pandas as pd
import numpy as np
import talib as ta
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
import matplotlib.pyplot as plt
import datetime
import random

from pypfopt import expected_returns, risk_models
from pypfopt import EfficientFrontier
from pypfopt import HRPOpt

In [17]:
# dataset = pd.read_csv('./IndexData.csv').set_index('Date')
dataset = pd.read_excel('./Bloomberg_DRL.xlsx', sheet_name = 'Data',index_col='Date')
dataset['UST_JPY'] = dataset['SPBDU10T Index'] * dataset['JPY BGN Curncy']
dataset['SPX_JPY'] = dataset['SPXT Index'] * dataset['JPY BGN Curncy']

In [18]:
target_name = ['UST_JPY', 'SPX_JPY', 'TPXDDVD Index', 'SPJGBTR Index']
feature_name = dataset.columns.drop(target_name)

In [19]:
dataset.head()

,SPX Index,SX5E Index,TPX Index,MXEF Index,SHCOMP Index,HSI Index,ASX Index,JPY BGN Curncy,EUR BGN Curncy,GBP BGN Curncy,...,IMM3CNCN Index,CESIEM Index,CESIJPY Index,CESIGL Index,SPBDU10T Index,SPXT Index,TPXDDVD Index,SPJGBTR Index,UST_JPY,SPX_JPY
Date,,,,,,,,,,,,,,,,,,,,,
Code,1.00,1.00,1.00,1.00,1.000,1.00,1.00,1.00,1.0000,1.0000,...,3,3.0,3.0,3.0,1.000,1.00,1.00,1.00,1.00000,1.0000
2003-01-03 00:00:00,908.59,2502.19,843.29,299.98,1319.868,9583.85,1923.75,119.63,1.0428,1.6113,...,11519,20.2,-11.8,4.4,296.102,1302.55,942.45,124.04,35422.68226,155824.0565
2003-01-10 00:00:00,927.57,2489.01,837.70,302.86,1384.859,9721.50,1911.39,119.32,1.0576,1.6076,...,13131,19.3,-4.2,3.6,294.035,1330.29,936.20,123.82,35084.25620,158730.2028
2003-01-17 00:00:00,901.78,2390.36,859.25,301.96,1478.675,9614.59,1844.77,117.83,1.0678,1.6131,...,24982,15.4,-6.5,2.3,296.949,1293.63,960.29,124.34,34989.50067,152428.4229
2003-01-24 00:00:00,861.40,2234.56,861.36,293.90,1479.070,9460.60,1749.49,117.82,1.0827,1.6333,...,31985,10.8,-13.6,9.1,299.355,1235.84,962.65,124.61,35270.00610,145606.6688


In [20]:
window = 52 * 3
StartDate = pd.to_datetime(datetime.datetime(2006, 12, 29)) - pd.offsets.Week(window)
EndDate = pd.to_datetime(datetime.datetime(2019, 12, 31)) 
# dataset = dataset[1:]
# dataset.index = pd.to_datetime(dataset.index)
# dataset = dataset[dataset.index.weekday==4]
df_data = dataset[target_name][1:][StartDate:EndDate]

In [21]:
df_data.head()

,UST_JPY,SPX_JPY,TPXDDVD Index,SPJGBTR Index
Date,,,,
2004-01-02 00:00:00,32546.88236,173228.4330,1179.90,122.62
2004-01-09 00:00:00,33137.58360,174953.3520,1204.88,122.82
2004-01-16 00:00:00,33242.22239,177493.0991,1197.19,123.67
2004-01-23 00:00:00,33180.46200,177980.0050,1217.49,123.36
2004-01-30 00:00:00,32715.13080,174710.0883,1184.26,123.69


In [22]:
mu = expected_returns.mean_historical_return(df_data, frequency=50)
S = risk_models.sample_cov(df_data)
ef = EfficientFrontier(mu, S)
ef.min_volatility()

{'UST_JPY': 0.0182545735719728,
 'SPX_JPY': 0.0129130509990589,
 'TPXDDVD Index': 0.0441875416387381,
 'SPJGBTR Index': 0.9246448337902302}

In [23]:
ef.efficient_return(target_return=0.05)

{'UST_JPY': 0.359672401012882,
 'SPX_JPY': 0.2296631891092218,
 'TPXDDVD Index': 0.0,
 'SPJGBTR Index': 0.410664409877896}

In [24]:
print(mu)
S

UST_JPY          0.050222
SPX_JPY          0.107719
TPXDDVD Index    0.066975
SPJGBTR Index    0.017527
dtype: float64


,UST_JPY,SPX_JPY,TPXDDVD Index,SPJGBTR Index
UST_JPY,0.036678,0.038779,0.027503,0.000156
SPX_JPY,0.038779,0.221957,0.136424,-0.007649
TPXDDVD Index,0.027503,0.136424,0.184611,-0.008534
SPJGBTR Index,0.000156,-0.007649,-0.008534,0.003248


In [25]:
df_data.pct_change().mean(axis=0)*50

UST_JPY          0.050222
SPX_JPY          0.107719
TPXDDVD Index    0.066975
SPJGBTR Index    0.017527
dtype: float64

In [26]:
ef.max_sharpe(risk_free_rate=0.01)

{'UST_JPY': 0.1305104808244413,
 'SPX_JPY': 0.0897910251988108,
 'TPXDDVD Index': 0.0174895030495256,
 'SPJGBTR Index': 0.7622089909272224}

In [45]:
weight = []
for w in range(1, len(df_data)-window): # 時点注意
    mu = expected_returns.mean_historical_return(df_data[w:w+window], frequency=52)
    S = risk_models.sample_cov(df_data[w:w+window])
    ef = EfficientFrontier(mu, S)
    weight.append(ef.min_volatility())

weight_df = pd.DataFrame([], columns=weight[0].keys())
weight_df = pd.concat([weight_df, pd.DataFrame.from_dict(weight)])
weight_df.index = df_data[window:-1].index
weight_df.to_csv('weight_GMV.csv')

In [46]:
weight_df.tail()

,UST_JPY,SPX_JPY,TPXDDVD Index,SPJGBTR Index
Date,,,,
2019-11-22 00:00:00,0.0,0.002288,0.039547,0.958165
2019-11-29 00:00:00,0.0,0.002925,0.039517,0.957558
2019-12-06 00:00:00,0.0,0.000205,0.041697,0.958098
2019-12-13 00:00:00,0.0,0.000233,0.041584,0.958183
2019-12-20 00:00:00,0.0,0.002299,0.039201,0.958500


In [47]:
weight = []
for w in range(1, len(df_data)-window): # 時点注意
    mu = expected_returns.mean_historical_return(df_data[w:w+window], frequency=52)
    S = risk_models.sample_cov(df_data[w:w+window])
    ef = EfficientFrontier(mu, S)
    weight.append(ef.max_sharpe(risk_free_rate=0.001))

weight_df = pd.DataFrame([], columns=weight[0].keys())
weight_df = pd.concat([weight_df, pd.DataFrame.from_dict(weight)])
weight_df.index = df_data[window:-1].index
weight_df.to_csv('weight_MaxSharpe.csv')

In [48]:
weight = []
for w in range(1, len(df_data)-window): # 時点注意
    mu = expected_returns.mean_historical_return(df_data[w:w+window], frequency=52)
    S = risk_models.sample_cov(df_data[w:w+window])
    ef = EfficientFrontier(mu, S)
    weight.append(ef.max_quadratic_utility(risk_aversion=5.0))

weight_df = pd.DataFrame([], columns=weight[0].keys())
weight_df = pd.concat([weight_df, pd.DataFrame.from_dict(weight)])
weight_df.index = df_data[window:-1].index
weight_df.to_csv('weight_MV.csv')

In [49]:
weight = []
df_data_pct = df_data.pct_change().dropna()
for w in range(len(df_data)-window-1): # 時点注意(HRPだけ0スタート)
    returns = df_data_pct[w:w+window]
    hrp = HRPOpt(returns)
    weight.append(hrp.optimize())

weight_df = pd.DataFrame([], columns=weight[0].keys())
weight_df = pd.concat([weight_df, pd.DataFrame.from_dict(weight)])
weight_df.index = df_data[window:-1].index
weight_df.to_csv('weight_HRP.csv')

In [59]:
weight = []
for w in range(1, len(df_data)-window): # 時点注意
    mu = expected_returns.mean_historical_return(df_data[w:w+window], frequency=52)
    S = risk_models.sample_cov(df_data[w:w+window])
    # print(w, mu, df_data.index[w+window])
    ef = EfficientFrontier(mu, S)
    try:
        weight.append(ef.efficient_return(0.05, market_neutral=False))
    except:
        try:
            weight.append(ef.efficient_return(0.03, market_neutral=False))
        except:
            weight.append(ef.efficient_return(0.01, market_neutral=False))

weight_df = pd.DataFrame([], columns=weight[0].keys())
weight_df = pd.concat([weight_df, pd.DataFrame.from_dict(weight)])
weight_df.index = df_data[window:-1].index
weight_df.to_csv('weight_TargetReturn.csv')